In [17]:
!pip install google-search-results

In [18]:
!sudo apt install pciutils lshw -y
!curl -fsSL https://ollama.com/install.sh | sh
!pip install -U langchain-ollama


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 122 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%#################################################                                 67.0%#########################################################            90.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [19]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [26]:
import threading
import subprocess
from time import sleep
import ollama
import time

In [27]:
def run_ollama():
    subprocess.run(["ollama", "serve"])

thread = threading.Thread(target=run_ollama)
thread.daemon = True
thread.start()
time.sleep(5)

2025/05/14 12:52:15 routes.go:1233: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICES: http_proxy: https_proxy:

In [28]:
!ollama pull llama3

[GIN] 2025/05/14 - 12:52:25 | 200 |      98.342µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ [GIN] 2025/05/14 - 12:52:26 | 200 |  512.325398ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕██████████████████▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕██████████████████▏  12 KB                         
pulling 8ab4849b038c: 100% ▕██████████████████▏  254 B                         
pulling 577073ffcc6c: 100% ▕██████████████████▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [29]:
import os
import threading
import subprocess
from time import sleep
import ollama
import time
import socket
from typing import List
import gradio as gr

from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.utilities import SerpAPIWrapper
from langchain_experimental.tools import PythonREPLTool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import AgentExecutor, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [30]:
MODEL_NAME = "llama3"
DOCS_DIR = "./langchain_docs"
FAISS_INDEX = "./faiss_index"
SERPAPI_API_KEY = "3a6824be60348a8f05cee889050277cce642d63593aa43aac0c64d3826751666"

In [31]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [32]:
def find_available_port(start_port=7860, end_port=7870):
    for port in range(start_port, end_port + 1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('localhost', port))
                return port
            except socket.error:
                continue
    raise OSError(f"No available ports between {start_port}-{end_port}")

PORT = find_available_port()

In [33]:
llm = Ollama(model=MODEL_NAME, temperature=0)
embeddings = OllamaEmbeddings(model=MODEL_NAME)

In [34]:
def setup_vectorstore():
    if os.path.exists(FAISS_INDEX):
        return FAISS.load_local(FAISS_INDEX, embeddings, allow_dangerous_deserialization=True)

    print("Building vector store...")
    urls = [
        "https://python.langchain.com/docs/get_started/introduction",
        "https://python.langchain.com/docs/modules/",
        "https://python.langchain.com/docs/guides/",
        "https://python.langchain.com/docs/ecosystem/integrations/",
        "https://python.langchain.com/docs/components/retrievers/",
        "https://python.langchain.com/docs/components/agents/",
        "https://python.langchain.com/docs/components/tools/"
    ]
    loader = WebBaseLoader(urls)
    docs = loader.load()
    
    print(len(docs))

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = splitter.split_documents(docs)
    
    print(len(splits))
    with open("langchain_chunks.txt", "w", encoding="utf-8") as f:
        for doc in splits:
             f.write(doc.page_content.strip() + "\n\n")

    vectorstore = FAISS.from_documents(splits, embeddings)
    vectorstore.save_local(FAISS_INDEX)
    return vectorstore
    
vectorstore = setup_vectorstore()
base_retriever = vectorstore.as_retriever()


compressor = LLMChainExtractor.from_llm(llm)

retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

In [35]:
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
python_repl = PythonREPLTool()

In [36]:
from langchain.agents import Tool

tools = [
    Tool(name="LangChainDocs", func=lambda q: str(retriever.invoke(q)), description="For LangChain documentation questions"),
    Tool(name="Search", func=search.run, description="For current events and web searches"),
    Tool(name="Python", func=python_repl.run, description="For executing Python code")
]

In [37]:
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    max_iterations=50, 
    max_execution_time=120  
)

/tmp/ipykernel_31/3309309059.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [38]:
def respond(message, chat_history):
    try:
        response_dict = agent_executor.invoke({"input": message})
        response = response_dict.get("output", "I couldn't generate a response.")
    except Exception as e:
        response = f"Error: {str(e)}"
    chat_history.append((message, response))
    return "", chat_history

In [43]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask me anything!")
    clear = gr.Button("Clear")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(server_port=7867)

/tmp/ipykernel_31/864294986.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7867
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://e09f21ce467887efbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...


time=2025-05-14T13:00:44.466Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-14T13:00:44.468Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z


[GIN] 2025/05/14 - 13:00:47 | 200 |  2.694568948s |       127.0.0.1 | POST     "/api/generate"
```json
{
  "action": "Search",
  "action_input": "Who is the current President of the United States?"
}
```

Note: I've used the Search tool to provide an answer that can be easily found through a web search. If you'd like me to provide more information or clarify the answer, please let me know!
Observation: Donald Trump
Thought:

time=2025-05-14T13:00:53.901Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-14T13:00:53.902Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z


[GIN] 2025/05/14 - 13:00:55 | 200 |  1.475932783s |       127.0.0.1 | POST     "/api/generate"
```json
{
  "action": "Final Answer",
  "action_input": "As of my knowledge cutoff, the current President of the United States is Joe Biden."
}
```

> Finished chain.
